In [1]:
import pandas as pd
import numpy as np
import sys
import os
import copy
import lightgbm as lgb
from sklearn.metrics import classification_report 
from sklearn.metrics import roc_auc_score

In [12]:
MainPath = os.path.normpath(os.getcwd() + os.sep + os.pardir)
DataPath = str(MainPath) + "\\Data\\"
print(DataPath)

C:\Users\Owner\Desktop\Projects\Research\Kaggle_Things\Data\


In [13]:
#datadir = "C:/Users\Owner\Desktop\Projects\Research\Kaggle_Things\Data"
# os.chdir("..")
# print (os.path.abspath(os.curdir))

In [25]:
trainAlldf = pd.read_csv(DataPath + "train.csv")
testdf = pd.read_csv(DataPath + "test.csv")

In [26]:
trainAlldf.describe()

,id,gravity,ph,osmo,cond,urea,calc,target
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,206.500000,1.017894,5.955459,651.545894,21.437923,278.657005,4.114638,0.444444
std,119.655756,0.006675,0.642260,234.676567,7.514750,136.442249,3.217641,0.497505
min,0.000000,1.005000,4.760000,187.000000,5.100000,10.000000,0.170000,0.000000
25%,103.250000,1.012000,5.530000,455.250000,15.500000,170.000000,1.450000,0.000000
50%,206.500000,1.018000,5.740000,679.500000,22.200000,277.000000,3.130000,0.000000
75%,309.750000,1.022000,6.280000,840.000000,27.600000,385.000000,6.630000,1.000000
max,413.000000,1.040000,7.940000,1236.000000,38.000000,620.000000,14.340000,1.000000


In [70]:
trainAlldf.corr()

,id,gravity,ph,osmo,cond,urea,calc,target
id,1.000000,-0.004775,-0.086619,0.008030,0.032843,-0.023822,0.032360,0.018222
gravity,-0.004775,1.000000,-0.290349,0.692317,0.470433,0.631710,0.494304,0.282577
ph,-0.086619,-0.290349,1.000000,-0.309495,-0.190185,-0.279749,-0.214402,-0.094983
osmo,0.008030,0.692317,-0.309495,1.000000,0.708480,0.809880,0.472114,0.244770
cond,0.032843,0.470433,-0.190185,0.708480,1.000000,0.499109,0.330609,0.172224
urea,-0.023822,0.631710,-0.279749,0.809880,0.499109,1.000000,0.489879,0.265211
calc,0.032360,0.494304,-0.214402,0.472114,0.330609,0.489879,1.000000,0.467439
target,0.018222,0.282577,-0.094983,0.244770,0.172224,0.265211,0.467439,1.000000


In [124]:
import matplotlib as plt
target_col = "target"
feature_cols = ['gravity', 'ph', 'osmo', 'cond', 'urea', 'calc']
for feature in feature_cols:
    plt.subplot

In [102]:
trainValRatio = 5
train_to_all = trainValRatio /(trainValRatio  + 1)
Ndata = len(trainAlldf)
traindf = trainAlldf.loc[0:int(Ndata*train_to_all)]
valdf = trainAlldf.loc[len(traindf):Ndata]
#display(traindf.tail(), valdf.head())

In [103]:
traindf.columns

Index(['id', 'gravity', 'ph', 'osmo', 'cond', 'urea', 'calc', 'target'], dtype='object')

In [104]:

y_train = traindf[target_col].values
y_val_true = valdf[target_col].values

X_train = traindf[feature_cols].values
X_val = valdf[feature_cols].values


In [123]:
import warnings
warnings.filterwarnings('ignore')
thold = 0.6
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
#lgb_eval = lgb.Dataset(X_, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    #'metric': {'l2', 'l1'},
    'max_depth':4,
    'num_leaves':12,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.5,
    'bagging_freq':5,
    'verbose': -1
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)
               # valid_sets=lgb_eval,
               # callbacks=[lgb.early_stopping(stopping_rounds=5)])

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_val_pred_float = gbm.predict(X_val, num_iteration=gbm.best_iteration)
y_val_pred = copy.deepcopy(y_val_pred_float)
y_val_pred[y_val_pred_float<thold]=0 
y_val_pred[y_val_pred_float>=thold]=1 
# eval

print(classification_report(y_val_true, y_val_pred))
print("ROC-Area test = ", roc_auc_score(y_val_true, y_val_pred_float))

y_train_pred_float = gbm.predict(X_train, num_iteration=gbm.best_iteration)
print("ROC-Area train = ",  roc_auc_score(y_train, y_train_pred_float))

Starting training...
Saving model...
Starting predicting...
              precision    recall  f1-score   support

           0       0.73      0.88      0.80        40
           1       0.75      0.54      0.63        28

    accuracy                           0.74        68
   macro avg       0.74      0.71      0.71        68
weighted avg       0.74      0.74      0.73        68

ROC-Area test =  0.7973214285714285
ROC-Area train =  0.8455296896086371


In [106]:
y_val_true.shape, y_val_pred.shape

((68,), (68,))

In [107]:
y_val_pred

array([0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])